In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content

In [2]:
load_dotenv(override=True)

True

# Part 1 - agents

In [3]:
instructions1 = "You are a sales agent working for ComplAI, a company that provides SaaS tools for \
    ensuring SOC2 compliance and preparing audits, powered by AI. You write professional, serious and \
    cold emails"
    
instructions2 = "You are a humorous, engaging sales agent working for ComplAI, a company that provides \
    SaaS tools for ensuring SOC2 compliance and preparing audits, powered by AI. You write witty, engaging \
    cold emails that are likely to get a response."
    
instructions3 = "You are a humorous, engaging sales agent working for ComplAI, a company that provides \
    SaaS tools for ensuring SOC2 compliance and preparing audits, powered by AI. You write concise, to the \
    point cold emails."

In [4]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model='gpt-4o-mini'
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model='gpt-4o-mini'
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model='gpt-4o-mini'
)

In [5]:
result = Runner.run_streamed(sales_agent1, input='Write a cold sales email')
async for event in result.stream_events():
    if event.type == 'raw_response_event' and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance Process with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. I am reaching out to introduce ComplAI, an innovative SaaS solution designed to simplify the SOC 2 compliance process through the power of AI.

As you may know, achieving and maintaining SOC 2 compliance can be a complex and resource-intensive task. ComplAI offers tools that automate many of the requirements, ensuring that your organization not only meets but exceeds compliance standards with confidence and efficiency.

Key benefits of using ComplAI include:
- **Automated Documentation**: Reduce the manual workload associated with gathering and managing compliance documentation.
- **Real-time Monitoring**: Leverage our AI-powered platform to ensure continuous compliance and be audit-ready at any moment.
- **Expert Guidance**: Access a library of resources and expert insights tailored to your specific compliance needs.

I would welcome the opportunity to discuss 

In [6]:
import asyncio
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )
    
outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify Your SOC 2 Compliance Process with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. 

Navigating the complexities of SOC 2 compliance can often be a daunting challenge. At ComplAI, we understand the unique hurdles that organizations face in this regard. Our state-of-the-art SaaS platform leverages artificial intelligence to streamline compliance processes and prepare for audits, ultimately saving you time and resources.

With ComplAI, you can expect:

- **Efficient Documentation:** Automated tracking and management of your compliance documentation.
- **Seamless Audits:** Simplified audit preparation that reduces the burden on your team.
- **Real-Time Insights:** Comprehensive analytics that keep you informed about your compliance status.

We would welcome the opportunity to discuss how ComplAI can specifically benefit your organization. Are you available for a brief conversation next week?

Thank you for considering this potential collaboration.


In [7]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
        Imagine you're a customer and pick the one you are the most likely to respond to. \
        Do not give any explanation, reply with the selected mail only."
)

In [8]:
message = 'Write a cold sales email'

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )
    
outputs = [result.final_output for result in results]

emails = "Cold sales emails: \n\n".join(outputs)
best = await Runner.run(sales_picker, emails)

print(f"Best sales email: {best.final_output}")

Best sales email: Subject: Ready to Level Up Your SOC 2 Game? 🎮

Hey [First Name],

I hope this email finds you surrounded by coffee and great ideas. Speaking of great ideas—how are your SOC 2 compliance efforts going? If they’re anything like my attempts at baking, you might be feeling a bit frazzled. 🥴

At ComplAI, we get it! Compliance can feel like trying to solve a Rubik’s cube blindfolded—confusing and a bit frustrating! But what if I told you it doesn’t have to be that way? Our AI-powered platform turns compliance headaches into smooth sailing, guiding you through audits like GPS for your SOC 2 journey. 🗺️✨

Here’s how we can help you:

- **Automated Document Management**: No more digging through files like a treasure hunt! 🏴‍☠️
- **Real-time Compliance Monitoring**: Get insights faster than you can find the best lunch spot! 🍔
- **Stress-Free Audit Prep**: Because who needs more stress in their life?

Ready to make compliance as easy as pie (or at least easier than baking it)? L

# Part 2 - tool use

In [9]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model='gpt-4o-mini'
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model='gpt-4o-mini'
)

sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model='gpt-4o-mini'
)

In [10]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [11]:
from loguru import logger
import requests

@function_tool
def push(
    message: str, 
    device: str = 'iphone', 
    timeout: int = 5) -> bool:
    """Send a push notification with given content, i.ex. a mail template.

    Args:
        message (str): Content of the message being sent
        device (str, optional): Name of the device to send notification to. Defaults to 'iphone'.
        timeout (int, optional): Timeout value. Defaults to 5.

    Returns:
        bool: True if succeeded, False on every other circumstance
    """
    if not all([pushover_user, pushover_token, pushover_url]):
        logger.error("Pushover configuration is missing.")
        return False

    logger.info("Sending push notification...")
    
    payload = {
        "user": pushover_user,
        "token": pushover_token,
        "message": message,
    }
    if device:
        payload["device"] = device

    try:
        response = requests.post(pushover_url, data=payload, timeout=timeout)
        if response.status_code != 200:
            logger.error(f"Pushover failed: {response.status_code} {response.text}")
            return False
        logger.info("Push notification sent successfully.")
        return True
    except requests.RequestException as e:
        logger.exception(f"Error sending push notification: {e}")
        return False

## Agent can also be converted to a tool

In [12]:
tool1 = sales_agent1.as_tool(
    tool_name='sales_agent1',
    tool_description='Write a cold sales email'
    )

In [13]:
# Use all Agents as tools
description = "Write a cold sales email"
tool1 = sales_agent1.as_tool(tool_name='sales_agent1', tool_description=description)
tool2 = sales_agent2.as_tool(tool_name='sales_agent2', tool_description=description)
tool3 = sales_agent3.as_tool(tool_name='sales_agent3', tool_description=description)

tools = [tool1, tool2, tool3, push]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x121cd3060>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x121cd2fc0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required'

# Sales Manager as planning agent

In [14]:
instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold \
    sales emails. You never generate emails yourself, always remember to use tools to do so. You try all the \
    tools that are available to choose the best possible email. At the end you pick the best one in your opinion \
    and send it to the user via push notification."
    
sales_manager = Agent(
    name='sales_manager',
    instructions=instructions,
    tools=tools,
    model='gpt-4o-mini'
)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace('Sales Manager'):
    result = await Runner.run(sales_manager, message)

2025-08-13 10:47:44.395 | INFO     | __main__:push:23 - Sending push notification...
2025-08-13 10:47:45.118 | INFO     | __main__:push:38 - Push notification sent successfully.


# Handoff

In [15]:
subject_instructions = "You can write a subject for a cold sales email. \
    You are given a message and you need to write a subject for an email that is \
    likely to get a response."
    
html_instruction = "You can convert a text email body to an HTML email body. \
    You are given a text email body which might have some Markdown formatting and \
    you need to convert it to an HTML email body with simple, clear, compelling \
    layout and design."

subject_writer = Agent(
    name='Email subject writer',
    instructions=subject_instructions,
    model='gpt-4o-mini'
)
subject_tool = subject_writer.as_tool(
    tool_name='subject_writer',
    tool_description='Write a subject for a cold sales email'
)

html_converter = Agent(
    name='HTML email body converter',
    instructions=html_instruction,
    model='gpt-4o-mini'
)
html_tool = html_converter.as_tool(
    tool_name="html_converter",
    tool_description="Convert a text email body to an HTML email body"
)

In [16]:
@function_tool
def push(
    message: str, 
    device: str = 'iphone', 
    timeout: int = 5) -> bool:
    """Send a push notification with given content, i.ex. a mail template.

    Args:
        message (str): Content of the message being sent
        device (str, optional): Name of the device to send notification to. Defaults to 'iphone'.
        timeout (int, optional): Timeout value. Defaults to 5.

    Returns:
        bool: True if succeeded, False on every other circumstance
    """
    if not all([pushover_user, pushover_token, pushover_url]):
        logger.error("Pushover configuration is missing.")
        return False

    logger.info("Sending push notification...")
    
    payload = {
        "user": pushover_user,
        "token": pushover_token,
        "message": message,
    }
    if device:
        payload["device"] = device

    try:
        response = requests.post(pushover_url, data=payload, timeout=timeout)
        if response.status_code != 200:
            logger.error(f"Pushover failed: {response.status_code} {response.text}")
            return False
        logger.info("Push notification sent successfully.")
        return True
    except requests.RequestException as e:
        logger.exception(f"Error sending push notification: {e}")
        return False

In [17]:
tools = [subject_tool, html_tool, push]

In [18]:
instructions = "You are an email formatter and sender. You receive the body of an email to be sent with push \
    notification. You first use the subject_writer tool to write a subject for the email, then use the html_converter \
    to convert the body to HTML format. Finally, you use the push tool to send a notification with created subject \
    and body in HTML."
    
push_agent = Agent(
    name='Push Manager',
    instructions=instructions,
    tools=tools,
    model='gpt-4o-mini',
    handoff_description='Convert and email to HTML and send it'
)

In [19]:
tools = [tool1, tool2 ,tool3]
handoffs = [push_agent]

In [20]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Push Manager agent to format and send the email via push notification."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

2025-08-13 10:48:20.112 | INFO     | __main__:push:20 - Sending push notification...
2025-08-13 10:48:20.891 | INFO     | __main__:push:35 - Push notification sent successfully.


# Guardrails

In [23]:
from pydantic import BaseModel

class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent(
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do",
    output_type=NameCheckOutput,
    model='gpt-4o-mini'
)

In [24]:
from agents import input_guardrail, GuardrailFunctionOutput

@input_guardrail
async def guardrail_against_name(ctx, agent, message) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(
        output_info={'found_name': result.final_output},
        tripwire_triggered=is_name_in_message
    )

In [25]:
careful_sales_manager = Agent(
    name='Careful Sales Manager',
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[push_agent],
    model='gpt-4o-mini',
    input_guardrails=[guardrail_against_name]
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace('Protected automated SDR'):
    result = await Runner.run(careful_sales_manager, message)

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire

In [26]:
careful_sales_manager = Agent(
    name='Careful Sales Manager',
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[push_agent],
    model='gpt-4o-mini',
    input_guardrails=[guardrail_against_name]
)

message = "Send out a cold sales email addressed to Dear CEO from the Head of Business Development"

with trace('Protected automated SDR'):
    result = await Runner.run(careful_sales_manager, message)

2025-08-13 11:06:04.479 | INFO     | __main__:push:20 - Sending push notification...
2025-08-13 11:06:06.819 | INFO     | __main__:push:35 - Push notification sent successfully.
